<a href="https://colab.research.google.com/github/NyanSwanAung/Pothole-Detection-using-MaskRCNN/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2 align=center> <b>MaskRCNN using Tensorflow Object Detection API (TF Version 2) </h2>

Using pre-trained model to identify potholes in live webcam and videos. The model was trained on [COCO2017](https://cocodataset.org/) and fine-tuned with pothole dataset from this [repo](https://github.com/SamdenLepcha/Pothole-Detection-With-Mask-R-CNN/tree/master/place_in_object_detection/images).

This pre-trained model is taken from [TensorFlow2 Object Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md)

## Prepare prerequisite

In [ ]:
#@title Download pretrained model from model zoo
import os
from IPython.display import clear_output

# Download using wget
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz

# Extract downloaded tar file 
!tar -xf mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz

os.mkdir('pre-trained-models')

# Rename extracted folder to pretrained-model
!mv mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8 pre-trained-models

clear_output()

In [ ]:
#@title Install TensorFlow Object Detection API
# Clone the tensorflow models repository
!git clone --depth 1 https://github.com/tensorflow/models

# # API Installation 
!sudo apt install -y protobuf-compiler
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

%cd ~/../content
clear_output()

In [ ]:
#@title Import dependencies
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
from object_detection.builders import model_builder

# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

import numpy as np
from PIL import Image
from IPython.display import display, Javascript, HTML

import os
import sys
import pathlib
from collections import defaultdict
from io import StringIO
from IPython.display import display


### Pothole Dataset

In [ ]:
#@title Download raw pothole dataset from github
!apt install subversion
!svn checkout https://github.com/SamdenLepcha/Pothole-Detection-With-Mask-R-CNN/trunk/place_in_object_detection/images
clear_output()

In [ ]:
#@title Download TF Record 
# download train.record
!wget https://raw.githubusercontent.com/SamdenLepcha/Pothole-Detection-With-Mask-R-CNN/master/place_in_object_detection/train.record

# download test.record
!wget https://raw.githubusercontent.com/SamdenLepcha/Pothole-Detection-With-Mask-R-CNN/master/place_in_object_detection/test.record

# download label map
!wget https://raw.githubusercontent.com/SamdenLepcha/Pothole-Detection-With-Mask-R-CNN/master/place_in_object_detection/training/label.pbtxt

clear_output()

# Remove unnecessary folders
!rm -r sample_data
!rm -f mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz

In [ ]:
#@title Make workspace

%cd ~/../content
os.mkdir('workspace')
os.mkdir('workspace/models')
os.mkdir('workspace/models/my_maskrcnn')

%cd ~/../content
os.mkdir('annotations')
!mv train.record label.pbtxt -t annotations 
!mv test.record valid.record
!mv valid.record -t annotations
!mv annotations pre-trained-models -t workspace
!cp /content/workspace/pre-trained-models/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/pipeline.config workspace/models/my_maskrcnn 

clear_output()

## Setup Paths

In [ ]:
WORKSPACE_PATH = 'workspace'
APIMODEL_PATH = 'models'

ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'

MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'

CUSTOM_MODEL_NAME = 'my_maskrcnn'

CONFIG_PATH = MODEL_PATH+'/' + CUSTOM_MODEL_NAME + '/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/' + CUSTOM_MODEL_NAME + '/'

print(CHECKPOINT_PATH )

workspace/models/my_maskrcnn/


## Config for fine tuning dataset

In [ ]:
#@title Download updated config from official repo

%cd /content/workspace/models/my_maskrcnn
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.config
!mv mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.config pipeline.config
%cd ~/../content

clear_output()

In [ ]:
# Load config file
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [ ]:
# Read config file and allow to rewrite access 
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:
  proto_str = f.read()
  text_format.Merge(proto_str, pipeline_config)

In [ ]:
# Rewriting 

# Hyperparameters 
pipeline_config.model.faster_rcnn.num_classes = 1
pipeline_config.train_config.batch_size = 1
pipeline_config.model.faster_rcnn.image_resizer.fixed_shape_resizer.height = 512
pipeline_config.model.faster_rcnn.image_resizer.fixed_shape_resizer.width = 512
pipeline_config.train_config.num_steps = 3000

# pre-trained model's checkpoint path
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/ckpt-0'

# pre-trained model's checkpoint type
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"

# Train label map path and record path 
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']

# Valid label map path and record path
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/valid.record']

# Shuffle valid dataset
pipeline_config.eval_input_reader[0].shuffle = True

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

## Train model

In [ ]:
# To see available arguments
!python models/research/object_detection/model_main_tf2.py --help

In [ ]:
# Run Train Script 
!python models/research/object_detection/model_main_tf2.py \
  --model_dir={CHECKPOINT_PATH} \
  --pipeline_config_path=workspace/models/{CUSTOM_MODEL_NAME}/pipeline.config \
  --alsologtostderr \
  --checkpoint_every_n=1000

2021-06-09 07:55:58.147350: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-06-09 07:56:00.164186: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-06-09 07:56:00.192367: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-06-09 07:56:00.192967: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-06-09 07:56:00.193029: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-06-09 07:56:00.195918: I tensorflow/stream_executor/platform/default

In [ ]:
# Run evaluation script 
!python models/research/object_detection/model_main_tf2.py \
  --model_dir={CHECKPOINT_PATH} \
  --pipeline_config_path=workspace/models/{CUSTOM_MODEL_NAME}/pipeline.config \
  --checkpoint_dir={CHECKPOINT_PATH}

## Export model to inference graph

In [ ]:
output_directory = CHECKPOINT_PATH + 'inference_graph'

!python models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {CHECKPOINT_PATH} \
    --output_directory {output_directory} \
    --pipeline_config_path {CONFIG_PATH}

2021-06-09 09:00:56.720339: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-06-09 09:00:59.056522: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-06-09 09:00:59.094163: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-06-09 09:00:59.094771: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-06-09 09:00:59.094819: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-06-09 09:00:59.113701: I tensorflow/stream_executor/platform/default